In [1]:
import regex as re
from bs4 import BeautifulSoup
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, WebDriverException, ElementNotInteractableException, TimeoutException, StaleElementReferenceException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys

import time
import math
import requests
import csv

from credentials import credential

In [2]:
class scrapeJobBank:
    def __init__(self, username, password, QUESTION1, ANSWER1, QUESTION2, ANSWER2, QUESTION3, ANSWER3, driverPath, jobTitle, location):
        self.username = username
        self.password = password
        self.driverPath = driverPath
        self.jobTitle = jobTitle
        self.location = location
        self.ANSWER1 = ANSWER1
        self.QUESTION1 = QUESTION1
        self.ANSWER2 = ANSWER2
        self.QUESTION2 = QUESTION2
        self.ANSWER3 = ANSWER3
        self.QUESTION3 = QUESTION3
    
    def init_driver(self):
        """Initializes instance of webdriver"""
        self.driver = webdriver.Chrome(executable_path=self.driverPath)
        self.driver.wait = WebDriverWait(self.driver, 10)
        return self.driver
    def login(self):
        """Logs into jobbank.gc.ca with given login credentials"""
        self.driver.get("https://www.jobbank.gc.ca/login")
        try:
            email_field = WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.ID, 'loginForm:input-email')))
            email_field.send_keys(self.username)
            time.sleep(1)
            login_button = WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.ID, 'loginForm:loginProcess')))
            login_button.click()
            time.sleep(5)
            try:
                pw_field = WebDriverWait(self.driver, 20).until(
                    EC.presence_of_element_located((By.ID, 'loginForm:input-password')))
                signin_button = WebDriverWait(self.driver, 20).until(
                    EC.presence_of_element_located((By.ID, 'loginStandardPlusUser')))
                pw_field.send_keys(self.password)
                time.sleep(1)
                signin_button.click()     
            except ElementNotInteractableException as e:
                print(str(e))
                
            try:
                answer_field = WebDriverWait(self.driver, 20).until(
                    EC.presence_of_element_located((By.ID, 'securityForm:input-security-answer')))       
                if self.driver.find_element_by_class_name("lead").text == self.QUESTION1:
                    answer_field.send_keys(self.ANSWER1)
                    time.sleep(1)
                    continue_button.click()
                elif self.driver.find_element_by_class_name("lead").text == self.QUESTION2:
                    answer_field.send_keys(self.ANSWER2)
                    time.sleep(1)
                    continue_button.click()
                elif self.driver.find_element_by_class_name("lead").text == self.QUESTION3:
                    answer_field.send_keys(self.ANSWER3)
                    time.sleep(1)
            except:
                differen_question_button = WebDriverWait(self.driver, 20).until(
                    EC.presence_of_element_located((By.ID, 'securityForm:j_id_3s_3_o')))
                different_question_button.click()
                
            continue_button = WebDriverWait(self.driver, 20).until(
                EC.presence_of_element_located((By.NAME, 'securityForm:j_id_2h_3_m')))
            print("Continue button found, awaiting click.")
            continue_button.click()

        except TimeoutException:
            print("TimeoutException on Login Method")
     
    def create_search(self):
        '''Search the jobbank for given titles and locations'''
        self.driver.get("https://www.jobbank.gc.ca/home")
        time.sleep(5)
        try:
            what_field = WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.ID, 'searchString')))            
            where_field = WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.ID, 'locationstring')))
            search_button = WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.ID, 'searchButton')))
            
            what_field.send_keys(self.jobTitle)
            what_field.send_keys(Keys.TAB)
            time.sleep(1)
            where_field.send_keys(self.location)
            search_button.click()
        except ElementNotInteractableException:
            print("Message: element not interactable")
        return self.driver   
    
    def perform_search(self):
        '''Returns a UTF-8 encoded html object as the result of a search'''
        found = int(self.driver.find_element_by_class_name("found").text.replace(',',''))
        pages = math.ceil(float(int(found)/25))
        for page in range(pages):
            try:
                self.driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
                more_results_button = WebDriverWait(self.driver, 10).until(
                    EC.presence_of_element_located((By.ID, 'moreresultbutton')))
                more_results_button.click()
                time.sleep(2)
            except (StaleElementReferenceException, TimeoutException):
                pass
        html = self.driver.page_source.encode('utf-8')
        soup = BeautifulSoup(html, 'html.parser')
        return soup
    
    def parser(self):
        soup = self.perform_search()
        articles = soup.find_all('article')
        header = ["Job_ID", "Title", "Date", "Company", "Location", "Salary", "Job Source", "Email"]
        job_descriptions = {header[0]:[],
                           header[1]:[],
                           header[2]:[],
                           header[3]:[],
                           header[4]:[],
                           header[5]:[],
                           header[6]:[],
                           header[7]:[]}
        counter = 0
        for article in articles:
            job_id = article['id'][-8:]
            title = article.find(name = 'span', class_ = 'noctitle').text.split('\n')[0]
            date = article.find(name = 'li', class_='date').text.split('\n')[0]
            business = article.find(name = 'li', class_="business").text
            location = article.find(name="li", class_="location").text.split('Location')[1].strip()
            if 'Salary not available' in article.find(name="li", class_="salary").text:
                salary = "Salary not available"
            else:
                salary = ' '.join(article.find(name="li", class_="salary").text.split()[2:])
            '''This is the application method handling. Each page has a how to apply button which must 
            be clicked to reveal the application pathway. Only applications via email are tracked'''
            self.driver.get("https://www.jobbank.gc.ca/jobsearch/jobposting/{}?source=searchresults".format(job_id))
            job_source = article.find(class_= 'wb-inv').text
            try:
                how_to_apply_button = WebDriverWait(self.driver, 5).until(
                    EC.presence_of_element_located((By.ID, 'applynowbutton')))
                how_to_apply_button.click()
                time.sleep(3)
                email = self.driver.find_element_by_xpath('//*[@id="howtoapply"]/p/a').text
            except (NoSuchElementException, TimeoutException):
                print("An external method is required at this step, i.e. jobillico, for the {} time".format(counter), end = "\r", flush = True)
                email = ''
                counter += 1

            job_descriptions[header[0]].append(job_id)
            job_descriptions[header[1]].append(title)
            job_descriptions[header[2]].append(date)
            job_descriptions[header[3]].append(business)
            job_descriptions[header[4]].append(location)
            job_descriptions[header[5]].append(salary)
            job_descriptions[header[6]].append(job_source)
            job_descriptions[header[7]].append(email)
       
        df = pd.DataFrame.from_dict(job_descriptions)
        with open('DailyResults.csv', 'a') as f:
            df.to_csv(f, encoding = 'utf-8_sig', header = False)
        self.driver.close()
        return df

In [3]:

scraper = scrapeJobBank(credential["USERNAME"], credential["PASSWORD"], 
                        credential["QUESTION1"], credential["ANSWER1"], 
                        credential["QUESTION2"], credential["ANSWER2"], 
                        credential["QUESTION3"], credential["ANSWER3"],
                        '/home/colton/anaconda3/bin/chromedriver', '', 'Lethbridge, AB')
scraper.init_driver()
scraper.login()
scraper.create_search()
scraper.parser()

TimeoutException on Login Method


,Job_ID,Title,Date,Company,Location,Salary,Job Source,Email
0,34802080,receptionist,"July 26, 2021",Allied Arts Council of Lethbridge,Lethbridge (AB),$18.90 hourly,Job Bank,hr@artslehbridge.org
1,34793727,Crisis counsellor,"July 23, 2021",YWCA Lethbridge and District,Lethbridge (AB),$24.04 hourly,Job Bank,hr@ywcalethbridge.com
2,34793274,Crisis counsellor,"July 23, 2021",YWCA Lethbridge and District,Lethbridge (AB),$24.04 hourly,Job Bank,hr@ywcalethbridge.com
3,34792588,general construction labourer,"July 23, 2021",Positive Life Concepts Lethbridge,Lethbridge (AB),$16.00 to $18.00 hourly (to be negotiated),Job Bank,http://www.positivelifeconcepts.com/
4,34788272,flexographic press operator,"July 22, 2021",CanCom Label & Packaging Ltd.,Lethbridge (AB),$27.28 to $28.50 hourly (to be negotiated),Job Bank,lou@cancomlabel.com
...,...,...,...,...,...,...,...,...
227,34370397,call centre agent - technical support,"April 30, 2021",HGS,"Fredericton, NB",$11.75 hourly,Job Bank,https://www.joinhgs.com/ca/current-openings/1712
228,34370367,customer service representative - call centre,"April 30, 2021",HGS,"Saint John, NB",$16.00 hourly,Job Bank,https://www.joinhgs.com/ca/current-openings/1713
229,34154078,cook,"April 06, 2021",Comex Food Production Inc.,"North Vancouver, BC",$15.50 hourly,Job Bank,hr@lataqueria.ca
230,34678864,"teacher, kindergarten","July 07, 2021",Horizon School Division,Barnwell (AB),Salary not available,Jobpostings.ca,
